# [3108. Minimum Cost Walk in Weighted Graph](https://leetcode.com/problems/minimum-cost-walk-in-weighted-graph/)

There is an undirected weighted graph with ```n``` vertices labeled from ```0``` to ```n - 1```.

You are given the integer ```n``` and an array ```edges```, where ```edges[i] = [ui, vi, wi]``` indicates that there is an edge between vertices ```ui``` and ```vi``` with a weight of ```wi```.

A walk on a graph is a sequence of vertices and edges. The walk starts and ends with a vertex, and each edge connects the vertex that comes before it and the vertex that comes after it. It's important to note that a walk may visit the same edge or vertex more than once.

The cost of a walk starting at node ```u``` and ending at node ```v``` is defined as the bitwise ```AND``` of the weights of the edges traversed during the walk. In other words, if the sequence of edge weights encountered during the walk is ```w0```, ```w1```, ```w2```, ```...```, ```wk```, then the cost is calculated as ```w0 & w1 & w2 & ... & wk```, where ```&``` denotes the bitwise ```AND``` operator.

You are also given a 2D array ```query```, where ```query[i] = [si, ti]```. For each query, you need to find the minimum cost of the walk starting at vertex ```si``` and ending at vertex ```ti```. If there exists no such walk, the answer is ```-1```.

Return the array ```answer```, where ```answer[i]``` denotes the minimum cost of a walk for query ```i```.



Example 1:
```python
Input: n = 5, edges = [[0,1,7],[1,3,7],[1,2,1]], query = [[0,3],[3,4]]

Output: [1,-1]

Explanation:


To achieve the cost of 1 in the first query, we need to move on the following edges: 0->1 (weight 7), 1->2 (weight 1), 2->1 (weight 1), 1->3 (weight 7).

In the second query, there is no walk between nodes 3 and 4, so the answer is -1.
```

Example 2:
```python
Input: n = 3, edges = [[0,2,7],[0,1,15],[1,2,6],[1,2,1]], query = [[1,2]]

Output: [0]

Explanation:


To achieve the cost of 0 in the first query, we need to move on the following edges: 1->2 (weight 1), 2->1 (weight 6), 1->2 (weight 1).
```


Constraints:

+ ```2 <= n <= 10^5```
+ ```0 <= edges.length <= 10^5```
+ ```edges[i].length == 3```
+ ```0 <= ui, vi <= n - 1```
+ ```ui != vi```
+ ```0 <= wi <= 10^5```
+ ```1 <= query.length <= 10^5```
+ ```query[i].length == 2```
+ ```0 <= si, ti <= n - 1```
+ ```si != ti```


### Disjoint Set (Union-Find) Solution

In [1]:
from typing import List

class Solution:
    def minimumCost(self, n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:
        def find(node):
            if parent[node] == -1:
                return node

            parent[node] = find(parent[node])
            
            return parent[node]

        def union(node1, node2):
            root1 = find(node1)
            root2 = find(node2)
            
            if root1 == root2:
                return
            
            if depth[root1] < depth[root2]:
                root1, root2 = root2, root1

            parent[root2] = root1

            if depth[root1] == depth[root2]:
                depth[root1] += 1

        parent = [-1] * n
        depth = [0] * n
        component_cost = [2**32 - 1] * n

        for node1, node2, _ in edges:
            union(node1, node2)
        
        for node1, _, weight in edges:
            root = find(node1)
            component_cost[root] &= weight
        
        answer = []

        for start, end in query:
            if find(start) != find(end):
                answer.append(-1)
            else:
                root = find(start)
                answer.append(component_cost[root])
        
        return answer

Run the code cell below to test the function.

In [2]:
sol = Solution()

print(sol.minimumCost(5, [[0,1,7],[1,3,7],[1,2,1]], [[0,3],[3,4]]))
print(sol.minimumCost(3, [[0,2,7],[0,1,15],[1,2,6],[1,2,1]], [[1,2]]))

[1, -1]
[0]


### Breadth-First Search (BFS) Solution

In [3]:
from collections import deque
from typing import List

class Solution:
    def minimumCost(self, n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:
        adj_list = [[] for _ in range(n)]

        for edge in edges:
            adj_list[edge[0]].append((edge[1], edge[2]))
            adj_list[edge[1]].append((edge[0], edge[2]))
        
        visited = [False] * n
        components = [0] * n
        component_cost = []
        component_id = 0

        for node in range(n):
            if not visited[node]:
                component_cost.append(
                    self._getComponentCost(
                        node,
                        adj_list,
                        visited,
                        components,
                        component_id)
                )
                component_id += 1
        
        answer = []

        for q in query:
            start, end = q
            if components[start] == components[end]:
                answer.append(component_cost[components[start]])
            else:
                answer.append(-1)
        
        return answer

    def _getComponentCost(
        self,
        source: int,
        adj_list: List[List[int]],
        visited: List[bool],
        components: List[int],
        component_id: int
    ) -> int:
        queue = deque()
        component_cost = 2**32 - 1

        queue.append(source)
        visited[source] = True

        while queue:
            node = queue.popleft()
            components[node] = component_id
            for next_node, weight in adj_list[node]:
                component_cost &= weight
                if not visited[next_node]:
                    visited[next_node] = True
                    queue.append(next_node)
        
        return component_cost

Run the code cell below to test the function.

In [4]:
sol = Solution()

print(sol.minimumCost(5, [[0,1,7],[1,3,7],[1,2,1]], [[0,3],[3,4]]))
print(sol.minimumCost(3, [[0,2,7],[0,1,15],[1,2,6],[1,2,1]], [[1,2]]))

[1, -1]
[0]


### Depth-First Search (DFS) Solution

In [5]:
from collections import deque
from typing import List

class Solution:
    def minimumCost(self, n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:
        adj_list = [[] for _ in range(n)]

        for edge in edges:
            adj_list[edge[0]].append((edge[1], edge[2]))
            adj_list[edge[1]].append((edge[0], edge[2]))
        
        visited = [False] * n
        components = [0] * n
        component_cost = []
        component_id = 0

        for node in range(n):
            if not visited[node]:
                component_cost.append(
                    self._getComponentCost(
                        node,
                        adj_list,
                        visited,
                        components,
                        component_id)
                )
                component_id += 1
        
        answer = []

        for q in query:
            start, end = q
            if components[start] == components[end]:
                answer.append(component_cost[components[start]])
            else:
                answer.append(-1)
        
        return answer

    def _getComponentCost(
        self,
        source: int,
        adj_list: List[List[int]],
        visited: List[bool],
        components: List[int],
        component_id: int
    ) -> int:
        visited[source] = True
        components[source] = component_id

        current_cost = 2**32 - 1

        for neighbour, weight in adj_list[source]:
            current_cost &= weight
            if not visited[neighbour]:
                current_cost &= self._getComponentCost(
                    neighbour,
                    adj_list,
                    visited,
                    components,
                    component_id
                )
        
        return current_cost

Run the code cell below to test the function.

In [6]:
sol = Solution()

print(sol.minimumCost(5, [[0,1,7],[1,3,7],[1,2,1]], [[0,3],[3,4]]))
print(sol.minimumCost(3, [[0,2,7],[0,1,15],[1,2,6],[1,2,1]], [[1,2]]))

[1, -1]
[0]
